In [1]:
import pandas as pd
#import torch
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
#import seaborn as sns

In [2]:
dfdemand = pd.read_csv('Demand_History.csv')
dfinfra = pd.read_csv('exisiting_EV_infrastructure_2018.csv')
sample_sub = pd.read_csv('sample_submission.csv')
dfdemand.head()

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563


In [3]:
def calculate_Distij(demand_point_x=dfdemand['x_coordinate'], 
                    demand_point_y=dfdemand['y_coordinate'],
                    supply_point_x = dfinfra['x_coordinate'],
                    supply_point_y=dfinfra['y_coordinate']): 
    # for x coord
    newdf = pd.DataFrame()
    lis = []
    for i in demand_point_x:
        for j in supply_point_x:
            lis.append(i-j)
    lisarr = np.asarray(lis)
    #print(lisarr)
    # for y coord
    lis1 = []
    for i in demand_point_y:
        for j in supply_point_y:
            lis1.append(i-j)
    lisarr1 = np.asarray(lis1)
    lis2 = []
    #Calculating linear distances between two points
    for x, y in zip(lisarr, lisarr1):
        lis2.append(np.sqrt(x**2 + y**2))
    Distij = np.asarray(lis2)
    #dist1 is the copy of the reshaped dist value
    Distij = Distij.reshape(4096, 100)
    #print(dist1.shape)
    return Distij

In [4]:
x = calculate_Distij()

In [5]:
def truncate_min_k(x, k, inplace=False):
    m, n = x.shape
    # get (unsorted) indices of top-k values
    mink_indices = np.argpartition(x, k, axis=1)[:, :k]
    # get k-th value
    rows, _ = np.indices((m, k))
    kth_vals = x[rows, mink_indices].max(axis=1)
    # get boolean mask of values smaller than k-th
    is_greater_than_kth = x > kth_vals[:, None]
    # replace mask by 0
    if not inplace:
        return np.where(is_greater_than_kth, 0, x)
    x[is_greater_than_kth] = 0
    return x

In [6]:
k=10
min_k = truncate_min_k(x, k=k)
min_k.shape

(4096, 100)

In [7]:
def summation_Dist():
    real_sum = []
    for i in range(4096):
        real_sum.append(min_k[i].sum())
        
    sum_dist = np.asarray(real_sum)[:, None]
    return sum_dist

In [8]:
sum_dist = summation_Dist()

In [9]:
def supply_demand(Dforecast=dfdemand['2018'].values[:, None]):  
    """
    Constraint5: 
    inputs:
    trunmin: This is the truncated minimum distances of each demand points from
    every supply points
    di: This is the forecasted demands for a particular year
    sum_dist: This is the sum of the truncated distances of each demand point from every 
    supply points reshaped into 4096 X 1 to match the broadcastable shape of 'trunmin'
    
    Return:
        demand satisfied by each supply point by the demand point
    """
    sum_dist = summation_Dist()
    min_k = truncate_min_k(x, k).reshape(4096, -1)    
    lis = []
    for j in range(4096):
        for distance in min_k[j]: #changed to minfive
            #for x, y, z in zip(min_sup, sum_dist[0], di[0]):
              #  lis.append((x*z)/y)
            lis.append((distance*Dforecast[j])/sum_dist[j])
 
    
    return np.asarray(lis)

In [10]:
DSij = supply_demand()
DSij.shape

(409600, 1)

In [11]:
dfinfra['scs_cap'] = dfinfra['existing_num_SCS']*200
dfinfra['fcs_cap'] = dfinfra['existing_num_FCS']*400
dfinfra['total_cap'] = dfinfra['scs_cap']+ dfinfra['fcs_cap']
dfinfra['parking_cap'] = (dfinfra['total_parking_slots'] - dfinfra['existing_num_SCS']) * 400 + (dfinfra['existing_num_SCS']) * 200
dfinfra['available_cap'] = dfinfra['parking_cap'] - dfinfra['total_cap']

In [12]:
dfdemand['2018'].shape

(4096,)

In [13]:
def add_infrastructure(Dforecast=dfdemand['2018'].values[:, None]):
    """
    purpose: This function calculates the number of additional infrastructure 
    needed at every(100) supply points over all the demand points.
    Particularly optimizing for more FCS if more infrastructure is
    needed
    inputs:
    sup_cap:
    This is the overall calculated supply capacity returned by our model(calculation parameters)
    scs_cap{constant}:
    This is the supply or charging capacity of slow charging station at every supply point
    const_total{constant}:
    This is the constant total charging capacity at every supply point
    """
    newval =  supply_demand(Dforecast)
    newval = newval.reshape(100, -1)
    DS = np.sum(newval, 1)
    smax= dfinfra['total_cap'].values
    lis = []# for fast charging stations
    for DSij, Smax in zip(DS, smax):
    
        result = Smax - DSij
        if result >= 0:
            lis.append(0)
        else:
            result = abs(result)
            lis.append((result//400) + 1) #+1 removed

        
    
    more_infra = np.asarray(lis) # capacity to add
    return more_infra

In [14]:
to_add = add_infrastructure()
to_add

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  6.,  0.,  0.,
        0.,  0.,  0.,  2.,  3.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  1.,  0.,  1.,  5.,  0.,  0.,  1.,  2.,  5.,  1.,  1.,  7.,
        2.,  1., 11.,  0.,  1., 11.,  1.,  6., 11.,  0.,  1.,  1.,  9.,
        9.,  0.,  0., 15.,  2., 11., 10.,  4., 11., 12.,  0.,  9., 10.,
        0., 17.,  0.,  2.,  8.,  2.,  4., 10.,  5., 17.,  0.,  9.,  6.,
        0.,  3.,  7.,  0.,  2.,  4.,  1.,  0.,  0.,  0.,  6.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [15]:
def excess_charging_cap(Dforecast=dfdemand['2018'].values[:, None], k=k):
    #dfinfra['new_FCS'] = dfinfra['existing_num_FCS'].values + to_add
    to_add = add_infrastructure(Dforecast)
    dfinfra['add_cap'] = to_add * 400
    dfinfra['excess_cap'] = dfinfra['available_cap'] - dfinfra['add_cap']
    #print(dfinfra['excess_cap'])
    excess=[]
    for i in dfinfra['excess_cap']:
        if i >= 0:
            excess.append(0)
        elif i < 0:
            excess.append(i)
    excess = np.asarray(excess)
    #print(len(excess))
    return excess

In [16]:
excess_charging_cap()

array([    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,  -800.,     0.,     0.,     0., -1200.,
           0.,     0.,  -400.,     0., -2400., -1600.,     0.,  -800.,
           0.,     0., -2800.,     0.,     0.,     0.,     0.,     0.,
           0.,     0., -4000.,     0., -2000.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.])

In [17]:
dfinfra['new_FCS'] = dfinfra['existing_num_FCS'].values + to_add
dfinfra['new_FCS_cap'] = dfinfra['new_FCS'] * 400
dfinfra['excess_charge_cap'] = dfinfra['parking_cap'] - (dfinfra['scs_cap'] + dfinfra['new_FCS_cap'])
dfinfra['excess_charge_cap'].values

array([ 6000.,  6400.,  4400.,  6400.,  6000.,  5200.,  3200.,  8000.,
        7200.,  4000.,   800.,  3200.,  4800.,  6400.,  5600.,  5200.,
        2000.,     0.,  6800.,  5600.,  3600.,  4000.,  4000.,  5200.,
        4000.,  4800.,  5200.,  6400.,  6000.,  7200.,  2800.,  7200.,
        6800.,  6000.,  4400.,   400.,  6400.,  1600.,  1600.,  6000.,
        4000.,  1200.,  6000.,  6400.,     0.,  6800.,  4400.,  1600.,
        4400.,  3600.,  5600.,  -800.,  3200.,  6400.,  7600., -1200.,
        4800.,     0.,  -400.,  3600., -2400., -1600.,  6400.,  -800.,
        2000.,  7200., -2800.,  4800.,  4000.,  4400.,  6800.,  4800.,
        1200.,  5200., -4000.,  6000., -2000.,  2000.,  6800.,  5600.,
         400.,  8400.,  2800.,  3600.,     0.,  4000.,  5600.,  5600.,
        2400.,  7200.,  4000.,  8400.,  4000.,  5200.,  5600.,  6800.,
        6800.,  5600.,  6400.,  4400.])

In [18]:
def adjusted_forecast(Dforecast=dfdemand['2018'].values[:, None], k=k):
    parking_cap = dfinfra['parking_cap'].values
    parking_cap.shape
    excess_cap = dfinfra['excess_charge_cap'].values
    excess = excess_charging_cap(Dforecast, k=k)
    newval =  supply_demand().reshape(100, -1)
    #print(newval)
    max_sup_cap = []
    
    for i in range(len(excess)):
        div_factor = excess[i]/(4096)
        #print(div_factor)
        max_sup_cap.append(newval[i] + div_factor)
    max_sup_cap = np.asarray(max_sup_cap)#.reshape(4096, -1)
    
    forecasted = []
    for i in range(4096):
        for cap in max_sup_cap:
            forecasted.append(cap[i])
    forecasted = np.asarray(forecasted)
    return forecasted

In [19]:
refined_forecast = adjusted_forecast()
refined_forecast.sum()

345529.6365968907

In [20]:
forecasty = np.sum(refined_forecast.reshape(4096, -1), 1)
forecasty.shape

(4096,)

In [21]:
infrastructure_to_add = add_infrastructure(Dforecast=forecasty[:, None])
print(infrastructure_to_add.sum())
infrastructure_to_add

175.0


array([3., 0., 0., 2., 0., 0., 0., 0., 4., 1., 6., 1., 2., 1., 0., 1., 5.,
       8., 0., 0., 0., 0., 4., 0., 3., 0., 0., 2., 0., 4., 5., 0., 0., 0.,
       6., 7., 0., 4., 2., 1., 0., 2., 0., 0., 3., 0., 0., 7., 0., 0., 0.,
       6., 2., 0., 0., 7., 0., 6., 2., 3., 5., 8., 0., 1., 5., 0., 6., 0.,
       0., 1., 0., 0., 3., 4., 7., 0., 6., 0., 0., 0., 0., 0., 0., 0., 5.,
       0., 0., 1., 7., 0., 0., 0., 2., 0., 0., 3., 0., 0., 1., 0.])

In [22]:
dfdemand['2018'].values[:, None].sum()

361529.6365968907

## Naive forecasting to test our model and constraint satisfaction

#### Here I assume that the demand for 2019 is exactly the same
as the demand for 2019 and that there is a +3 increase in all demand 
values 

#### Forecasting for 2019

The forecast is basically the 2018 data that I used as forecast to evaluate 
the constraints. All we need to do is to use the result in the refined forecast above and the result in infrastructure_to_add 
above, for adding infrastructures

In [23]:
infrastructure_to_add = infrastructure_to_add.reshape(-1, 100)
add_fcs_nineteen = infrastructure_to_add # since I optimized for fcs
new_fcs_cap1 = dfinfra['existing_num_FCS'].values + add_fcs_nineteen
sub_arr = sample_sub
print(sub_arr['value'].values[100:200].sum())
sub_arr['value'].values[100:200] = new_fcs_cap1
print(sub_arr['value'].values[100:200].sum())

619.0
794.0


For demand values for 2019:

In [24]:
print(sub_arr['value'].values[200:409800].sum())
sub_arr['value'].values[200:409800] = refined_forecast
print(sub_arr['value'].values[200:409800].sum())

361589.5405005043
345529.6365968907


#### Forecasting for 2020

In [25]:
twenty_forecast = dfdemand['2018'] + 3
k=10

In [26]:
supply_demand(twenty_forecast.values[:, None])


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [27]:
dfinfra['scs_cap'] = dfinfra['existing_num_SCS']*200
dfinfra['fcs_cap'] = dfinfra['existing_num_FCS']*400
dfinfra['total_cap'] = dfinfra['scs_cap']+ dfinfra['fcs_cap']
dfinfra['parking_cap'] = (dfinfra['total_parking_slots'] - dfinfra['existing_num_SCS']) * 400 + (dfinfra['existing_num_SCS']) * 200
dfinfra['available_cap'] = dfinfra['parking_cap'] - dfinfra['total_cap']

In [28]:
add_infrastructure(Dforecast=twenty_forecast.values[:, None])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  6.,  0.,  0.,
        0.,  0.,  0.,  3.,  3.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  2.,  0.,  2.,  5.,  0.,  0.,  1.,  3.,  5.,  1.,  1.,  7.,
        2.,  1., 12.,  1.,  1., 11.,  2.,  6., 11.,  0.,  1.,  1.,  9.,
        9.,  0.,  0., 16.,  2., 11., 10.,  4., 11., 13.,  0.,  9., 10.,
        0., 17.,  0.,  2.,  8.,  2.,  4., 10.,  5., 18.,  0.,  9.,  6.,
        0.,  3.,  7.,  0.,  3.,  4.,  1.,  0.,  0.,  0.,  6.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [29]:
excess_charging_cap(Dforecast=twenty_forecast.values[:, None], k=k)

array([    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,  -800.,     0.,     0.,     0., -1600.,
           0.,     0.,  -400.,     0., -2400., -2000.,     0.,  -800.,
           0.,     0., -2800.,     0.,     0.,     0.,     0.,     0.,
           0.,     0., -4400.,     0., -2000.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.])

In [30]:
dfinfra['new_FCS'] = dfinfra['existing_num_FCS'].values + to_add
dfinfra['new_FCS_cap'] = dfinfra['new_FCS'] * 400
dfinfra['excess_charge_cap'] = dfinfra['parking_cap'] - (dfinfra['scs_cap'] + dfinfra['new_FCS_cap'])
dfinfra['excess_charge_cap'].values

array([ 6000.,  6400.,  4400.,  6400.,  6000.,  5200.,  3200.,  8000.,
        7200.,  4000.,   800.,  3200.,  4800.,  6400.,  5600.,  5200.,
        2000.,     0.,  6800.,  5600.,  3600.,  4000.,  4000.,  5200.,
        4000.,  4800.,  5200.,  6400.,  6000.,  7200.,  2800.,  7200.,
        6800.,  6000.,  4400.,   400.,  6400.,  1600.,  1600.,  6000.,
        4000.,  1200.,  6000.,  6400.,     0.,  6800.,  4400.,  1600.,
        4400.,  3600.,  5600.,  -800.,  3200.,  6400.,  7600., -1200.,
        4800.,     0.,  -400.,  3600., -2400., -1600.,  6400.,  -800.,
        2000.,  7200., -2800.,  4800.,  4000.,  4400.,  6800.,  4800.,
        1200.,  5200., -4000.,  6000., -2000.,  2000.,  6800.,  5600.,
         400.,  8400.,  2800.,  3600.,     0.,  4000.,  5600.,  5600.,
        2400.,  7200.,  4000.,  8400.,  4000.,  5200.,  5600.,  6800.,
        6800.,  5600.,  6400.,  4400.])

In [31]:
twenty_forecast.values.sum()

373817.6365968907

In [32]:
adjusted_forecast(Dforecast=twenty_forecast.values[:, None], k=k)

array([0.        , 0.        , 5.85623275, ..., 0.        , 0.6789755 ,
       0.        ])

In [33]:
adjusted_forecast()

array([0.        , 0.        , 5.85623275, ..., 0.        , 0.6789755 ,
       0.        ])

In [34]:
refined_forecast_twenty = adjusted_forecast(Dforecast=twenty_forecast.values[:, None], k=k)

In [35]:
forecasty2 = np.sum(refined_forecast_twenty.reshape(4096, -1), 1)
forecasty2.shape

(4096,)

In [36]:
forecasty2

array([ 41.38746297,  98.8781994 , 124.19859965, ...,  54.37862105,
        40.87988231,  92.08054368])

In [37]:
infrastructure_to_add2 = add_infrastructure(Dforecast=forecasty2[:, None])
print(infrastructure_to_add2.sum())
infrastructure_to_add2

173.0


array([3., 0., 0., 2., 0., 0., 0., 0., 4., 1., 6., 1., 2., 1., 0., 1., 5.,
       8., 0., 0., 0., 0., 4., 0., 3., 0., 0., 2., 0., 4., 5., 0., 0., 0.,
       5., 7., 0., 4., 2., 1., 0., 2., 0., 0., 3., 0., 0., 7., 0., 0., 0.,
       6., 1., 0., 0., 7., 0., 6., 2., 3., 5., 8., 0., 1., 5., 0., 6., 0.,
       0., 1., 0., 0., 3., 4., 7., 0., 6., 0., 0., 0., 0., 0., 0., 0., 5.,
       0., 0., 1., 7., 0., 0., 0., 2., 0., 0., 3., 0., 0., 1., 0.])

In [38]:
print(sample_sub['value'].values[409900:410000])

[ 3.  7. 14.  5.  6.  6.  8.  6.  1.  5.  0.  4.  5.  5. 12.  6.  2.  0.
  8.  8.  8.  8.  4.  7.  3. 10. 11.  4.  6.  3.  2.  9. 10.  7.  1.  1.
  9.  4.  4.  6.  6.  4.  4. 10.  6.  6.  8.  1. 10. 13.  8.  1.  7. 16.
 10.  1.  9.  1.  3.  3.  3.  0. 10.  5.  2. 11.  2. 13.  9.  6.  8.  7.
  1.  4.  1. 15.  2. 11.  8.  6.  9.  7.  7.  6.  2.  7.  7.  5.  1. 10.
  5. 12.  3.  6.  9.  4. 10. 14.  5. 11.]


In [39]:
infrastructure_to_add2 = infrastructure_to_add2.reshape(-1, 100)
add_fcs_twenty = infrastructure_to_add2 # since I optimized for fcs
print(dfinfra['existing_num_FCS'])
new_fcs_cap2 = dfinfra['existing_num_FCS'].values + add_fcs_twenty
print(new_fcs_cap2)
sub_arr['value'].values[409900:410000] = new_fcs_cap2
print(sub_arr['value'].values[409900:410000])

0      3
1      7
2     14
3      5
4      6
      ..
95     4
96    10
97    14
98     5
99    11
Name: existing_num_FCS, Length: 100, dtype: int64
[[ 6.  7. 14.  7.  6.  6.  8.  6.  5.  6.  6.  5.  7.  6. 12.  7.  7.  8.
   8.  8.  8.  8.  8.  7.  6. 10. 11.  6.  6.  7.  7.  9. 10.  7.  6.  8.
   9.  8.  6.  7.  6.  6.  4. 10.  9.  6.  8.  8. 10. 13.  8.  7.  8. 16.
  10.  8.  9.  7.  5.  6.  8.  8. 10.  6.  7. 11.  8. 13.  9.  7.  8.  7.
   4.  8.  8. 15.  8. 11.  8.  6.  9.  7.  7.  6.  7.  7.  7.  6.  8. 10.
   5. 12.  5.  6.  9.  7. 10. 14.  6. 11.]]
[ 6.  7. 14.  7.  6.  6.  8.  6.  5.  6.  6.  5.  7.  6. 12.  7.  7.  8.
  8.  8.  8.  8.  8.  7.  6. 10. 11.  6.  6.  7.  7.  9. 10.  7.  6.  8.
  9.  8.  6.  7.  6.  6.  4. 10.  9.  6.  8.  8. 10. 13.  8.  7.  8. 16.
 10.  8.  9.  7.  5.  6.  8.  8. 10.  6.  7. 11.  8. 13.  9.  7.  8.  7.
  4.  8.  8. 15.  8. 11.  8.  6.  9.  7.  7.  6.  7.  7.  7.  6.  8. 10.
  5. 12.  5.  6.  9.  7. 10. 14.  6. 11.]


In [40]:
sub_arr['value'].values.sum()

709845.1770973953

In [41]:
sample_sub['value'].values.sum()

709845.1770973953

#### For demand values in 2020

In [42]:
print(sub_arr['value'].values[410000:].sum())

361589.5405005043


In [43]:
sub_arr['value'].values[410000:] = refined_forecast
print(sub_arr['value'].values[410000:].sum())

345529.6365968907


In [44]:
dfinfra.head()

,supply_point_index,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS,scs_cap,fcs_cap,total_cap,parking_cap,available_cap,add_cap,excess_cap,new_FCS,new_FCS_cap,excess_charge_cap
0,0,50.163110,19.412014,23,5,3,1000,1200,2200,8200,6000,0.0,6000.0,3.0,1200.0,6000.0
1,1,37.336451,58.119225,27,4,7,800,2800,3600,10000,6400,0.0,6400.0,7.0,2800.0,6400.0
2,2,46.709232,57.525650,31,6,14,1200,5600,6800,11200,4400,0.0,4400.0,14.0,5600.0,4400.0
3,3,30.528626,55.379835,26,5,5,1000,2000,3000,9400,6400,0.0,6400.0,5.0,2000.0,6400.0
4,4,51.521781,35.116755,32,11,6,2200,2400,4600,10600,6000,0.0,6000.0,6.0,2400.0,6000.0


In [45]:
sub_arr[100:]

,year,data_type,demand_point_index,supply_point_index,value
100,2019,FCS,NaN,0,6.000000
101,2019,FCS,NaN,1,7.000000
102,2019,FCS,NaN,2,14.000000
103,2019,FCS,NaN,3,7.000000
104,2019,FCS,NaN,4,6.000000
...,...,...,...,...,...
819595,2020,DS,4095.0,95,0.000000
819596,2020,DS,4095.0,96,0.000000
819597,2020,DS,4095.0,97,0.000000
819598,2020,DS,4095.0,98,0.678976


In [60]:
dfinfra['total_slot_19'] = sub_arr['value'][:100] + sub_arr['value'][100:200] 
dfinfra['total_slot_20'] = sub_arr['value'][409800:409900] + sub_arr['value'][409900:410000] 

In [71]:
#Double checking for constraint 3
dfinfra.head()
print(dfinfra.where(dfinfra['total_parking_slots'] < dfinfra['total_slot_19']).dropna(axis=0))
print(dfinfra.where(dfinfra['total_parking_slots'] < dfinfra['total_slot_20']).dropna(axis=0))

Empty DataFrame
Columns: [supply_point_index, x_coordinate, y_coordinate, total_parking_slots, existing_num_SCS, existing_num_FCS, scs_cap, fcs_cap, total_cap, parking_cap, available_cap, add_cap, excess_cap, new_FCS, new_FCS_cap, excess_charge_cap, total_slot_19, total_slot_20]
Index: []
Empty DataFrame
Columns: [supply_point_index, x_coordinate, y_coordinate, total_parking_slots, existing_num_SCS, existing_num_FCS, scs_cap, fcs_cap, total_cap, parking_cap, available_cap, add_cap, excess_cap, new_FCS, new_FCS_cap, excess_charge_cap, total_slot_19, total_slot_20]
Index: []


In [48]:
# Double checking to ensure adherence to constraint 3
for i in range(len(sub_arr['value'])):
    if sub_arr['value'][i] < 0:
        sub_arr['value'][i] = 0

C:\Users\BABATUNDE\AppData\Local\Temp\ipykernel_7548\2951237892.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_arr['value'][i] = 0


In [65]:
sub_arr.head()

,year,data_type,demand_point_index,supply_point_index,value
0,2019,SCS,NaN,0,5.0
1,2019,SCS,NaN,1,4.0
2,2019,SCS,NaN,2,6.0
3,2019,SCS,NaN,3,5.0
4,2019,SCS,NaN,4,11.0


In [69]:
sub_arr['value'].head()

0     5.0
1     4.0
2     6.0
3     5.0
4    11.0
Name: value, dtype: float64

In [66]:
sub_arr.to_csv('eighty.csv')

In [67]:
df = pd.read_csv('eighty.csv')
df

,Unnamed: 0,year,data_type,demand_point_index,supply_point_index,value
0,0,2019,SCS,NaN,0,5.000000
1,1,2019,SCS,NaN,1,4.000000
2,2,2019,SCS,NaN,2,6.000000
3,3,2019,SCS,NaN,3,5.000000
4,4,2019,SCS,NaN,4,11.000000
...,...,...,...,...,...,...
819595,819595,2020,DS,4095.0,95,0.000000
819596,819596,2020,DS,4095.0,96,0.000000
819597,819597,2020,DS,4095.0,97,0.000000
819598,819598,2020,DS,4095.0,98,0.678976


In [52]:
for i in range(len(df['value'])):
    if df['value'][i] < 0:
        df['value'][i] = 0

C:\Users\BABATUNDE\AppData\Local\Temp\ipykernel_7548\2988834060.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['value'][i] = 0


In [53]:
df['value'].min()

0.0

In [54]:
#df.to_csv('seventh1')